PIPELINE DE EXTRACCIÓN DE DATOS

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import time 
import pandas as pd
import random, string
from selenium.webdriver.common.by import By
import re

In [2]:
driver = webdriver.Chrome(executable_path='/home/florian/inteligencia-artificial/publicar-investigacion/nlp-as-comentarios-restaurantes-unmsm/chromedriver')
#driver.get("https://www.google.com.pe/maps/search/cevicheria/@-12.0626704,-77.029618,13z/data=!3m1!4b1?hl=es-419");

/tmp/ipykernel_138461/1814558355.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/home/florian/inteligencia-artificial/publicar-investigacion/nlp-as-comentarios-restaurantes-unmsm/chromedriver')


Obtener la estructura de la pagina web página incial

In [7]:
def saveFile(title, cordenate, stars, comments):
    frame = pd.DataFrame({'titulo' : title,'coordenada' : cordenate,'calificacion' : stars, 'comentario' : comments})
    file_name = title+"-"+str(random.randint(10000, 100000))+".csv"
    frame.to_csv(file_name, index = None, header=True,encoding='utf-8')

def filterQuantity(quantity):
    pattern = r'[(,)]'
    return int(re.sub(pattern,"", quantity)) 
    

In [8]:
def scrollDown(elemento, numberOfScrollDowns):
    while numberOfScrollDowns >=0:
        elemento.send_keys(Keys.PAGE_DOWN)
        numberOfScrollDowns -= 1
        time.sleep(3)
    #return browser

PROCESO DE SCRAPING

In [6]:
driver.get("https://www.google.com/maps/search/resturantest+chorrillos/@-12.0773039,-77.1696105,11z/data=!4m2!2m1!6e5");

In [9]:
#inputElement = driver.find_element_by_id("searchboxinput")
#inputElement.send_keys('piso 21')
page_header = BeautifulSoup(driver.page_source)
list_restaurant = page_header.find_all('div', { 'class': 'Nv2PK THOPZb CpccDe' })
i = 0
for restaurant in list_restaurant:
    try:
        time.sleep(10)
        #page_header = BeautifulSoup(driver.page_source)
        title = restaurant.find("div", {"class": "qBF1Pd fontHeadlineSmall"}).text
        print('============== ', title, ' ======================')
        quantity = restaurant.find("span", { "class": "UY7F9" }).text
        quantity_reviews = filterQuantity(quantity)
        
        if(quantity_reviews > 4000):
            elem = int(quantity_reviews / 3) - 500
        elif quantity_reviews > 1000:
            elem = int(quantity_reviews / 3) - 100
        else:
            elem = int(quantity_reviews / 3)
    
        elemento = driver.find_elements(By.CLASS_NAME,'Nv2PK')
        elemento = elemento[i]
        elemento.click()
        time.sleep(10)
        bottonClick = driver.find_elements(By.XPATH, "//button[@class='hh2c6 ']")
        bottonClick = bottonClick[0]
        bottonClick.click()
        time.sleep(10)
        #print(BeautifulSoup(driver.page_source))
        elementScroll = driver.find_elements(By.XPATH, "//div[@class='m6QErb DxyBCb kA9KIf dS8AEf ']")
        #print(len(elementScroll), "##########################")
        scrollDown(elementScroll[0], elem)
        
        currentURL = driver.current_url;
        cordenate = currentURL[currentURL.find('@')+1:currentURL.find('z')+1]
        
        page_reviews = BeautifulSoup(driver.page_source)
        calification = []
        comments = []
        for item in page_reviews.find_all('div', { 'class': 'jJc9Ad' }):
            try:
                textSpan = item.find('span', { 'class': 'wiI7pd'}).text
            except:
                textSpan = None
            if textSpan is not None:
                comments.append(textSpan)
                calification.append(len(item.find_all('img', {'class' : 'hCCjke vzX5Ic'})))
        
        #comments = [item.text is None if "" else item.text for item in page_reviews.find_all('span', {'class' : 'wiI7pd'})]
        #stars = [ for item in page_reviews.find_all('span', {'class' : 'kvMYJc'})]
        saveFile(title, cordenate, calification, comments)
        #closeWindow = driver.find_element(By.XPATH, "//button[@class='VfPpkd-icon-LgbsSe yHy1rc eT1oJ mN1ivc']")
        #closeWindow.click()
        print('============== FIN ======================')
    except OSError as err:
        print("error ->", err)
        #break
    except NoSuchElementException as err:
        print("error ->", err)
        #break
    except ElementNotVisibleException as err:
        ##exception for page empty
        print("error ->", err)
        #break
    except:
        print("error ->", "Exception general")
    i = i + 1

==============  El Hornero Chorrillos  ======================
============== FIN ======================
==============  Dos Peces Restaurante  ======================
============== FIN ======================
==============  Restaurant SONIA  ======================
============== FIN ======================
==============  El Paraíso Marino Restaurante | Cevichería  ======================
============== FIN ======================
==============  D'Garden Restaurant cevicheria  ======================
============== FIN ======================
